In [ ]:
from LibraryENGcopia import *
from tqdm.notebook import tqdm
import sklearn.preprocessing as ps
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from random import randint
from fastdtw import fastdtw
import copy
name_data = '2018-11-27T10-29-42POP 2 BL .h5'
name_data2 = '2018-11-27T12-03-02POP 2 stimulation.h5'
name_data3='2018-11-27T10-56-39MiP5 KA stimulation.h5'
complete_string='/Users/Gaia_1/Downloads/PoiProMEA-master/h5files/'+name_data
complete_string2='/Users/Gaia_1/Downloads/PoiProMEA-master/h5files/'+name_data2
complete_string3='/Users/Gaia_1/Downloads/PoiProMEA-master/h5files/'+name_data3

In [ ]:
#data = h5py.File(complete_string,'r')
#data = h5py.File(complete_string2,'r')
data = h5py.File(complete_string3,'r')

data_readings = data['Data']['Recording_0']['AnalogStream']['Stream_0']['ChannelData'][()]
info = data['Data']['Recording_0']['AnalogStream']['Stream_0']['InfoChannel'][()]
info_table = pd.DataFrame(info, columns = list(info.dtype.fields.keys()))
labels = info_table['Label']
readings = pd.DataFrame(data = data_readings.transpose(), columns = labels)
fs = 10000 #Sampling Frequency
print(readings.shape)

In [ ]:
inizio=0
len_data=64000 #192 secondi
#len_data=int(len(readings)/2)
prova_multi=readings.iloc[inizio:len_data, :17]
#prova=prova.drop([b'Ref'],axis=1)
prova=prova_multi[b'46']
ref=readings[b'Ref']
ref=ref[inizio:len_data]
#prova=readings.iloc[:150000, :5]
print(prova.shape,ref.shape)

In [ ]:
lowcut = 300
highcut = 3000
fs=10000
order=8
b,a=butter_bandpass(lowcut,highcut,fs,order=order)
filt_ref=filtfilt(b,a,ref)
filt_prova=filtfilt(b,a,prova)
f_prova = filt_prova - filt_ref
prova=f_prova
prova.shape
r_prova=prova.reshape(-1, 1)
scaler = StandardScaler()
s_prova = scaler.fit_transform(r_prova)
prova=s_prova
s_prova.shape

In [ ]:
pos, neg=find_all_spikes(prova)

firing rate baseline: 90
firing rate pop 2 stimulation: 87
firing rate pop 5 KA stimulation: 110

In [ ]:
len(pos)

In [ ]:
pos_cut,n_pos, neg_cut,n_neg = cut(pos,neg,prova)
savedp = copy.deepcopy(pos_cut)
savedn = copy.deepcopy(neg_cut)

In [ ]:
pos_cut=mask_cuts(pos_cut)
neg_cut= mask_cuts(neg_cut)

In [ ]:
x=randint(1,len(pos_cut))

plt.plot(savedp[x])
plt.plot(pos_cut[x])
plt.show()
plt.plot(savedn[x])
plt.plot(neg_cut[x])
plt.show()

In [ ]:
final_data_pos= clus(pos_cut,'PCA','kmeans',n_pos,4,len_data)

In [ ]:
plt.hist(np.diff(final_data_pos[0]),bins=100, density=True, alpha=0.5, color='blue', edgecolor='black')

baseline: 2 cluster (10, 9 firing rates)
pop 2 stimulation: 2 cluster (19, 17 firing rates)

In [ ]:
final_data_neg= clus(neg_cut,'PCA','kmeans',n_neg,5,len_data)

In [ ]:
len(onecut)

In [ ]:
#manual clustering based on DTW distance:
together=[]
to_remove=[]
orig=copy.deepcopy(neg_cut)
onecut=neg_cut
left=onecut
#for i in range(len(onecut)):
i=1
while i < len(onecut):
    to_remove.append(i)
    for j in range(i+1,100):#len(onecut)):
        print(i,j)
        spike1=onecut[i]
        spike2=onecut[j]
        distance,path=fastdtw(spike1,spike2)
        if distance<5.5:
            print('y')
            inds=(ind[i]-1,ind[j]-1)
            together.append(inds)
            #print(together)
            to_remove.append(j)
            #print(to_remove)
            i+=1
            k=i
            while k<le000in(onecut):
                print(k)
                pre=k-1
                inds=(ind[pre],ind[k]) 
                if inds in together:
                    i+=1
left=np.delete(left,to_remove)

together

In [ ]:
i=1
print(i)
#for i in range(len(onecut)):
nex=i+1
print(nex)
inds=(ind[i],ind[nex])
print(inds)
if inds in together:
    print(inds, 'in')
    i+=1
print(i)


In [ ]:
together

In [ ]:
#let's verify the dtw similarity within the identified clusters:
onecut=neg_cut
ind=n_neg
cluster=final_data_neg[4]

lt=len(cluster)*len(cluster)
pbar = tqdm(total = lt)
saved=[]
for k in cluster:
    for i in range(len(onecut)):
        if ind[i]==k:
            for j in range(i + 1, len(onecut)):
                if ind[j] in cluster:
                    spike1 = onecut[i]
                    spike2 = onecut[j]
                    pbar.update(1)
                    # Calculate DTW distance and alignment path
                    distance, path = fastdtw(spike1, spike2)
                    #if distance<20:
                    s=(ind[i],ind[j])
                    saved.append(s)
                    plt.figure()
                    plt.plot(spike1)
                    plt.plot(spike2)
                    plt.show()
                    print(f"DTW distance between spike {i} {ind[i]},{k} and spike {j} {ind[j]}: {distance}")
                    print(f"Alignment path: {path}")
    saved

baseline: 2 cluster (8,9 firing rate)
pop 2 stimulation: 2 cluster (24, 26 firing rate)

In [ ]:
plt.hist(np.diff(final_data_neg[2]),bins=100, density=True, alpha=0.5, color='blue', edgecolor='black')

In [ ]:
final_data_neg[2]

from fastdtw import fastdtw
i=14
j=17
spike1=pos_cut[i]
spike2=pos_cut[j]
plt.plot(spike1)
plt.plot(spike2)
distance, path = fastdtw(spike1, spike2)

print(f"DTW distance between spike {i+1} and spike {j+1}: {distance}")
print(f"Alignment path: {path}")

In [ ]:
x=randint(1,len(pos_cut))
print(x)

plt.plot(pos_cut[x])

In [ ]:
onecut=neg_cut
ind=n_neg
cluster=final_data_neg[2]
saved=[]

l=8
k=cluster[l]

i+=1
print(ind[i],k)
if ind[i]==k:
    print('proceed')

    j=i
    
    spike1 = onecut[i]
    spike2 = onecut[j]


In [ ]:
for j in range(i + 1, len(onecut)):
    spike1 = onecut[i]
    spike2 = onecut[j]

    distance, path = fastdtw(spike1, spike2)
    if distance<6:
        s=(ind[i],ind[j])
        saved.append(s)
        print(f"DTW distance between spike {i} {ind[i]},{k} and spike {j} {ind[j]}: {distance}")
        if ind[j] in cluster:
            print(ind[j],' cointained in cluster')
        else:
            plt.figure()
            plt.plot(spike1)
            plt.plot(spike2)
            plt.show()


In [ ]:
#compute dtw for spikes 570 and 688 (not in same cluster)
#688 876
#1092 1095 1593 2304 5503 12507 125012
#1577 1801 2009 3202 4213 5211 8226
# 3070 11117 12665 16296 22891 24702 28775 30217
# 3124 51485 53426 

In [ ]:
for p,m in enumerate(final_data_neg):
    for o,n in enumerate(m):
        if n==570:
            print('ind', o,'value',n, 'cluster', m, 'number',p)

In [ ]:
final_data_neg[4]

In [ ]:
onecut=neg_cut
ind=n_neg
cluster=final_data_neg[4]

lt=len(onecut)*len(onecut)
pbar = tqdm(total = lt)
saved=[]
# Iterate through pairs of spikes and apply DTW
for k in cluster:
    for i in range(len(onecut)):
        if ind[i]==k:
            for j in range(i + 1, len(onecut)):
                spike1 = onecut[i]
                spike2 = onecut[j]
                pbar.update(1)
                # Calculate DTW distance and alignment path
                distance, path = fastdtw(spike1, spike2)
                if distance<20:
                    s=(ind[i],ind[j])
                    saved.append(s)
                    plt.figure()
                    plt.plot(spike1)
                    plt.plot(spike2)
                    plt.show()
                    print(f"DTW distance between spike {i} {ind[i]},{k} and spike {j} {ind[j]}: {distance}")
                    print(f"Alignment path: {path}")
saved

In [ ]:
final_data_pos[0]

In [ ]:
final_data_pos[1]

In [ ]:
plt.plot(pos_cut[0])

In [ ]:
final_data_pos[0]

In [ ]:
lt=len(final_data_pos[0])*len(final_data_pos[0])
pbar = tqdm(total = lt)
# Iterate through pairs of spikes and apply DTW
for i in final_data_pos[0]:
    for j in final_data_pos[0]:
        spike1 = pos_cut[i]
        spike2 = pos_cut[j]
        pbar.update(1)
        # Calculate DTW distance and alignment path
        distance, path = fastdtw(spike1, spike2)
        if distance<20:
            plt.figure()
            plt.plot(spike1)
            plt.plot(spike2)
            plt.show()
            print(f"DTW distance between spike {i+1} and spike {j+1}: {distance}")
            print(f"Alignment path: {path}")